In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import multivariate_normal as mvn

from armored.models import *
from armored.preprocessing import *

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

# set plot parameters
params = {'legend.fontsize': 18,
          'figure.figsize': (16, 12),
          'lines.linewidth': 4,
          'axes.labelsize': 24,
          'axes.titlesize':24,
          'axes.linewidth':5,
          'xtick.labelsize':20,
          'ytick.labelsize':20}
plt.rcParams.update(params)
plt.style.use('seaborn-colorblind')
plt.rcParams['pdf.fonttype'] = 42

np.random.seed(123)

# Set parameters of script

In [2]:
# levels of richness to sample 
richnesses = [5, 10, 15]

# number of times to run k-fold x-validation 
n_trials = 5

# each trial uses different random partitioning of data
n_splits = 10

# import data

In [3]:
df_main = pd.read_csv("Data/reactor_ubiome_sparsity.csv")
df_main.head()

,Experiments,Time,s1,s2,s3,s4,s5,s6,s7,s8,...,rf2,rf3,rf4,rf5,rf6,rf7,rf8,rf9,rf10,feed
0,Richness_5_0,0.0,0.010393,0.0,0.0,0.0,0.0,0.0,0.009889,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,Richness_5_0,26.0,0.147226,0.0,0.0,0.0,0.0,0.0,0.097496,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,Richness_5_0,52.0,0.304041,0.0,0.0,0.0,0.0,0.0,0.197555,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,Richness_5_0,78.0,0.250334,0.0,0.0,0.0,0.0,0.0,0.193828,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,Richness_5_0,104.0,0.221226,0.0,0.0,0.0,0.0,0.0,0.147685,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [6]:
df

,Experiments,Time,s1,s2,s3,s4,s5,s6,s7,s8,...,rf2,rf3,rf4,rf5,rf6,rf7,rf8,rf9,rf10,feed
600,Richness_15_100,0.0,0.010561,0.010254,0.010699,0.010848,0.010331,0.009680,0.010422,0.000000,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
601,Richness_15_100,26.0,0.072397,0.063815,0.056233,0.074568,0.038432,0.049793,0.054025,0.000000,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
602,Richness_15_100,52.0,0.081855,0.079769,0.059377,0.095614,0.038967,0.052279,0.059937,0.000000,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
603,Richness_15_100,78.0,0.069072,0.069458,0.047642,0.080828,0.029121,0.041477,0.047187,0.000000,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
604,Richness_15_100,104.0,0.057386,0.051678,0.041291,0.063873,0.022549,0.033584,0.034257,0.000000,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,Richness_15_149,26.0,0.066998,0.066948,0.051907,0.000000,0.037732,0.047021,0.000000,0.051890,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
896,Richness_15_149,52.0,0.086376,0.079451,0.059121,0.000000,0.041766,0.059172,0.000000,0.060808,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
897,Richness_15_149,78.0,0.078227,0.058031,0.052112,0.000000,0.033834,0.048203,0.000000,0.054796,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
898,Richness_15_149,104.0,0.061190,0.053966,0.041454,0.000000,0.028809,0.038395,0.000000,0.042612,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


### Define which variables are species and which are metabolites

In [4]:
# specify species and metabolite names 
species = df_main.columns.values[2:-12]
metabolites =  []
controls = []
system_variables = np.concatenate((np.array(species), np.array(metabolites)))
system_variables

array(['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11',
       's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20'],
      dtype=object)

In [5]:
# all experiment names in original dataframe
all_exp_names = df_main.Experiments.values

# evaluation times 
t_eval = np.unique(df_main.Time.values)

# perform k-fold x-validation several times (n_trials) with randomized partitions of data
for trial in range(n_trials):
    for richness in richnesses:

        # pull dataframe with experiments of a particular richness 
        rich_inds = [f"Richness_{richness}" in exp_name for exp_name in all_exp_names]
        df = df_main.iloc[rich_inds].copy()

        # get exp names at particular richness 
        rich_exp_names = df.Experiments.values
        unique_experiments = np.unique(df.Experiments.values)

        # initialize dataframe to save predictions
        df_kfold_mirnn = pd.DataFrame()
        df_kfold_rnn = pd.DataFrame()

        # object to get indices of kfold splits
        kf = KFold(n_splits=n_splits, random_state=trial, shuffle=True)

        # loop over k-fold splits
        for train_index, test_index in kf.split(unique_experiments):

            # update index values to pull trajectories 
            train_index = np.in1d(rich_exp_names, unique_experiments[train_index])
            test_index  = np.in1d(rich_exp_names, unique_experiments[test_index])

            ### MIRNN ### 

            # pull train and test dataframes
            train_df  = df.iloc[train_index].copy() 
            test_df   = df.iloc[test_index].copy() 

            # format data
            train_data, exp_names, N = format_data(train_df, species, metabolites, controls)
            test_data, test_exp_names, _ = format_data(test_df, species, metabolites, controls)

            # transform data
            scaler = ZeroMaxScaler().fit(train_data)
            train_data = scaler.transform(train_data)
            test_data  = scaler.transform(test_data)

            # Apply RNN to predict metabolite concentration on sparsely sampled data:
            brnn = miRNN(n_species=len(species), n_metabolites=len(metabolites), n_controls=0, n_hidden=16, N=N)
            brnn.fit(train_data)

            # Make predictions
            preds, stdvs, COV = brnn.predict(test_data)

            # Transform values back to original scale
            preds = scaler.inverse_transform(preds)
            stdvs = scaler.inverse_transform_stdv(stdvs)
            test_data = scaler.inverse_transform(test_data)

            # save data
            pred_df_kfold = pd.DataFrame()
            for i, exp_name in enumerate(test_exp_names):
                pred_df = pd.DataFrame()
                pred_df['Experiments'] = [exp_name]*len(t_eval)
                pred_df['Time'] = t_eval
                for j, variable in enumerate(system_variables):
                    pred_df[variable + " true"] = test_data[i, :, j]
                    pred_df[variable + " pred"] = preds[i, :, j]
                    pred_df[variable + " stdv"] = stdvs[i, :, j]
                pred_df_kfold = pd.concat((pred_df_kfold, pred_df))
            df_kfold_mirnn = pd.concat((df_kfold_mirnn, pred_df_kfold))

            ### RNN ### 

            # pull train and test dataframes
            train_df  = df.iloc[train_index].copy() 
            test_df   = df.iloc[test_index].copy() 

            # format data
            train_data, exp_names, N = format_data(train_df, species, metabolites, controls)
            test_data, test_exp_names, _ = format_data(test_df, species, metabolites, controls)

            # transform data
            scaler = ZeroMaxScaler().fit(train_data)
            train_data = scaler.transform(train_data)
            test_data  = scaler.transform(test_data)

            # Apply RNN to predict metabolite concentration on sparsely sampled data:
            brnn = RNN(n_species=len(species), n_metabolites=len(metabolites), n_controls=0, n_hidden=16, N=N)
            brnn.fit(train_data)

            # Make predictions
            preds, stdvs, COV = brnn.predict(test_data)

            # Transform values back to original scale
            preds = scaler.inverse_transform(preds)
            stdvs = scaler.inverse_transform_stdv(stdvs)
            test_data = scaler.inverse_transform(test_data)

            # save data
            pred_df_kfold = pd.DataFrame()
            for i, exp_name in enumerate(test_exp_names):
                pred_df = pd.DataFrame()
                pred_df['Experiments'] = [exp_name]*len(t_eval)
                pred_df['Time'] = t_eval
                for j, variable in enumerate(system_variables):
                    pred_df[variable + " true"] = test_data[i, :, j]
                    pred_df[variable + " pred"] = preds[i, :, j]
                    pred_df[variable + " stdv"] = stdvs[i, :, j]
                pred_df_kfold = pd.concat((pred_df_kfold, pred_df))
            df_kfold_rnn = pd.concat((df_kfold_rnn, pred_df_kfold))

            # save kfold predictions
            df_kfold_mirnn.to_csv(f"Fig2/Sparsity/MiRNN_richness_{richness}_trial_{trial}.csv", index=False)
            df_kfold_rnn.to_csv(f"Fig2/Sparsity/RNN_richness_{richness}_trial_{trial}.csv", index=False)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Epoch: 0, Loss: 348.69943, Residuals: -13.16558, Convergence:   inf
Epoch: 10, Loss: 292.63345, Residuals: -11.95849, Convergence: 0.017567
Epoch: 20, Loss: 246.23839, Residuals: -10.85993, Convergence: 0.017285
Epoch: 30, Loss: 207.75134, Residuals: -9.86046, Convergence: 0.017032
Epoch: 40, Loss: 175.65509, Residuals: -8.95042, Convergence: 0.016845
Epoch: 50, Loss: 148.74039, Residuals: -8.12124, Convergence: 0.016718
Epoch: 60, Loss: 126.08158, Residuals: -7.36590, Convergence: 0.016621
Epoch: 70, Loss: 106.98752, Residuals: -6.67825, Convergence: 0.016491
Epoch: 80, Loss: 90.94470, Residuals: -6.05284, Convergence: 0.016271
Epoch: 90, Loss: 77.51767, Residuals: -5.48471, Convergence: 0.015954
Epoch: 100, Loss: 66.30173, Residuals: -4.96907, Convergence: 0.015593
Epoch: 110, Loss: 56.92589, Residuals: -4.50107, Convergence: 0.015160
Epoch: 120, Loss: 49.10813, Residuals: -4.07654, Convergence: 0.014651
Epoch: 130, Loss: 42.58258, Residuals: -3.69185, Convergence: 0.014096
Epoch: 14

Epoch: 960, Loss: 501.41833, Residuals: 0.00584, Convergence: 0.001154
Epoch: 970, Loss: 495.22577, Residuals: 0.00595, Convergence: 0.001169

Epoch: 973, Evidence: 2460.44727, Convergence: 0.003077
Epoch: 980, Loss: 495.26544, Residuals: 0.00581, Convergence: 0.000992

Epoch: 980, Evidence: 2467.70068, Convergence: 0.002939
Epoch: 980, Loss: 501.45584, Residuals: 0.00581, Convergence:   inf
Epoch: 990, Loss: 494.80399, Residuals: 0.00546, Convergence: 0.001133

Epoch: 993, Evidence: 2473.43701, Convergence: 0.002319
Epoch: 1000, Loss: 494.71899, Residuals: 0.00514, Convergence: 0.000998

Epoch: 1000, Evidence: 2476.96436, Convergence: 0.001424
Epoch: 1000, Loss: 499.72546, Residuals: 0.00514, Convergence:   inf

Epoch: 1009, Evidence: 2481.49854, Convergence: 0.001827
Epoch: 1010, Loss: 498.15704, Residuals: 0.00452, Convergence: 0.001367

Epoch: 1013, Evidence: 2484.06006, Convergence: 0.001031
Epoch: 1020, Loss: 495.96155, Residuals: 0.00370, Convergence: 0.000838

Epoch: 1020, Evid


Epoch: 699, Evidence: 459.68408, Convergence: 0.020005
Epoch: 700, Loss: 497.59485, Residuals: 0.02963, Convergence: 0.001186
Epoch: 710, Loss: 492.13840, Residuals: 0.02950, Convergence: 0.001026

Epoch: 711, Evidence: 467.21799, Convergence: 0.016125
Epoch: 720, Loss: 492.46967, Residuals: 0.02916, Convergence: 0.001046

Epoch: 724, Evidence: 475.69995, Convergence: 0.017830
Epoch: 730, Loss: 492.89581, Residuals: 0.02870, Convergence: 0.001087

Epoch: 736, Evidence: 482.77826, Convergence: 0.014662
Epoch: 740, Loss: 493.04742, Residuals: 0.02806, Convergence: 0.001118

Epoch: 748, Evidence: 491.00348, Convergence: 0.016752
Epoch: 750, Loss: 493.15399, Residuals: 0.02733, Convergence: 0.001146

Epoch: 758, Evidence: 497.51245, Convergence: 0.013083
Epoch: 760, Loss: 492.58691, Residuals: 0.02647, Convergence: 0.001136
Epoch: 770, Loss: 487.44019, Residuals: 0.02554, Convergence: 0.000990

Epoch: 770, Evidence: 504.18896, Convergence: 0.013242
Epoch: 770, Loss: 492.37518, Residuals: 

Epoch: 330, Loss: 10.61762, Residuals: -0.52076, Convergence: 0.001944
Epoch: 340, Loss: 10.42461, Residuals: -0.47240, Convergence: 0.001749
Epoch: 350, Loss: 10.25564, Residuals: -0.42843, Convergence: 0.001537
Epoch: 360, Loss: 10.10812, Residuals: -0.38854, Convergence: 0.001376
Epoch: 370, Loss: 9.98081, Residuals: -0.35257, Convergence: 0.001205
Epoch: 380, Loss: 9.87040, Residuals: -0.32002, Convergence: 0.001050

Epoch: 385, Evidence: -63.66090, Convergence:   inf
Epoch: 390, Loss: 22.12826, Residuals: -0.28971, Convergence: 0.003647
Epoch: 400, Loss: 21.38900, Residuals: -0.26152, Convergence: 0.003227
Epoch: 410, Loss: 20.75599, Residuals: -0.23594, Convergence: 0.002844
Epoch: 420, Loss: 20.21142, Residuals: -0.21272, Convergence: 0.002503
Epoch: 430, Loss: 19.74151, Residuals: -0.19159, Convergence: 0.002221
Epoch: 440, Loss: 19.33472, Residuals: -0.17242, Convergence: 0.001973
Epoch: 450, Loss: 18.97441, Residuals: -0.15501, Convergence: 0.001788
Epoch: 460, Loss: 18.65624

...

Epoch: 410, Loss: 115.56813, Residuals: -0.21786, Convergence: 0.001008

Epoch: 411, Evidence: 4708.15869, Convergence: 0.644092
Epoch: 420, Loss: 456.19409, Residuals: -0.19742, Convergence: 0.001702
Epoch: 430, Loss: 449.09549, Residuals: -0.17875, Convergence: 0.001475
Epoch: 440, Loss: 443.03577, Residuals: -0.16172, Convergence: 0.001275
Epoch: 450, Loss: 437.84744, Residuals: -0.14618, Convergence: 0.001109

Epoch: 458, Evidence: 6722.22998, Convergence: 0.299614
Epoch: 460, Loss: 981.76459, Residuals: -0.13215, Convergence: 0.001309
Epoch: 470, Loss: 969.97791, Residuals: -0.11953, Convergence: 0.001129

Epoch: 479, Evidence: 7680.95996, Convergence: 0.124819
Epoch: 480, Loss: 1343.20300, Residuals: -0.10794, Convergence: 0.001086

Epoch: 489, Evidence: 8000.93506, Convergence: 0.039992
Epoch: 490, Loss: 1502.10645, Residuals: -0.09765, Convergence: 0.001024

Epoch: 492, Evidence: 8119.23438, Convergence: 0.014570

Epoch: 494, Evidence: 8166.41211, Convergence: 0.005777

Epoch: 


Epoch: 497, Evidence: 5212.34033, Convergence: 0.001196
Epoch: 500, Loss: 1620.77783, Residuals: -0.01127, Convergence: 0.001128

Epoch: 501, Evidence: 5222.01611, Convergence: 0.001853

Epoch: 505, Evidence: 5225.90674, Convergence: 0.000744
Epoch: 510, Loss: 1609.97229, Residuals: -0.00247, Convergence: 0.001469

Epoch: 513, Evidence: 5240.62646, Convergence: 0.003551
Epoch: 520, Loss: 1603.06567, Residuals: 0.00217, Convergence: 0.000977

Epoch: 520, Evidence: 5260.25293, Convergence: 0.003731
Epoch: 520, Loss: 1615.92822, Residuals: 0.00217, Convergence:   inf

Epoch: 527, Evidence: 5277.51318, Convergence: 0.003271
Epoch: 530, Loss: 1606.65576, Residuals: 0.00296, Convergence: 0.001362

Epoch: 532, Evidence: 5292.61768, Convergence: 0.002854

Epoch: 539, Evidence: 5306.62109, Convergence: 0.002639
Epoch: 540, Loss: 1608.96960, Residuals: 0.00293, Convergence: 0.001174

Epoch: 543, Evidence: 5317.86816, Convergence: 0.002115

Epoch: 547, Evidence: 5326.18164, Convergence: 0.001561

Epoch: 80, Loss: 565.53320, Residuals: -5.25047, Convergence: 0.010343
Epoch: 90, Loss: 511.80411, Residuals: -4.77420, Convergence: 0.009519
Epoch: 100, Loss: 467.69467, Residuals: -4.34312, Convergence: 0.008651
Epoch: 110, Loss: 431.65216, Residuals: -3.95262, Convergence: 0.007705
Epoch: 120, Loss: 402.04083, Residuals: -3.59944, Convergence: 0.006739
Epoch: 130, Loss: 377.89539, Residuals: -3.27954, Convergence: 0.005795
Epoch: 140, Loss: 357.84894, Residuals: -2.99028, Convergence: 0.005121
Epoch: 150, Loss: 341.42838, Residuals: -2.72844, Convergence: 0.004538
Epoch: 160, Loss: 327.84933, Residuals: -2.49192, Convergence: 0.003863
Epoch: 170, Loss: 316.81284, Residuals: -2.27766, Convergence: 0.003150
Epoch: 180, Loss: 307.49829, Residuals: -2.08467, Convergence: 0.002811
Epoch: 190, Loss: 299.95856, Residuals: -1.90976, Convergence: 0.002376
Epoch: 200, Loss: 293.67224, Residuals: -1.75172, Convergence: 0.002018
Epoch: 210, Loss: 288.48969, Residuals: -1.60873, Convergence: 0.0

Epoch: 290, Loss: 28.28608, Residuals: -0.70753, Convergence: 0.002284
Epoch: 300, Loss: 27.70121, Residuals: -0.63949, Convergence: 0.001961
Epoch: 310, Loss: 27.21268, Residuals: -0.57801, Convergence: 0.001671
Epoch: 320, Loss: 26.80540, Residuals: -0.52246, Convergence: 0.001405
Epoch: 330, Loss: 26.46305, Residuals: -0.47221, Convergence: 0.001226
Epoch: 340, Loss: 26.17606, Residuals: -0.42675, Convergence: 0.000979

Epoch: 340, Evidence: 1720.30945, Convergence:   inf
Epoch: 340, Loss: 132.21530, Residuals: -0.42675, Convergence:   inf
Epoch: 350, Loss: 128.56688, Residuals: -0.38594, Convergence: 0.002599
Epoch: 360, Loss: 125.56125, Residuals: -0.34888, Convergence: 0.002190
Epoch: 370, Loss: 123.07356, Residuals: -0.31530, Convergence: 0.001853
Epoch: 380, Loss: 121.00927, Residuals: -0.28484, Convergence: 0.001557
Epoch: 390, Loss: 119.28461, Residuals: -0.25736, Convergence: 0.001347
Epoch: 400, Loss: 117.84008, Residuals: -0.23252, Convergence: 0.001133

Epoch: 408, Eviden

Epoch: 460, Loss: 1617.64990, Residuals: -0.10075, Convergence: 0.000974

Epoch: 460, Evidence: 5226.82568, Convergence: 0.002364
Epoch: 460, Loss: 1630.32239, Residuals: -0.10075, Convergence:   inf

Epoch: 464, Evidence: 5231.67188, Convergence: 0.000926
Epoch: 470, Loss: 1611.83740, Residuals: -0.07141, Convergence: 0.001403

Epoch: 474, Evidence: 5253.43945, Convergence: 0.005066
Epoch: 480, Loss: 1611.01123, Residuals: -0.04570, Convergence: 0.000974

Epoch: 480, Evidence: 5273.91650, Convergence: 0.003883
Epoch: 480, Loss: 1624.37952, Residuals: -0.04570, Convergence:   inf

Epoch: 482, Evidence: 5277.59570, Convergence: 0.000697
Epoch: 490, Loss: 1603.08240, Residuals: -0.01775, Convergence: 0.001190

Epoch: 493, Evidence: 5293.39990, Convergence: 0.003681

Epoch: 498, Evidence: 5316.14160, Convergence: 0.004278
Epoch: 500, Loss: 1612.22632, Residuals: -0.00330, Convergence: 0.001030

Epoch: 504, Evidence: 5332.00684, Convergence: 0.002975
Epoch: 510, Loss: 1604.76465, Residuals